In [27]:
import pandas as pd
import joblib
from datetime import datetime

models = joblib.load('sales_forecasting_arima_models.pkl')

forecast_steps = 7
forecast_results = []

for item_id, model_fit in models.items():
    forecast = model_fit.get_forecast(steps=forecast_steps)
    forecast_index = pd.date_range(start=datetime.now(), periods=forecast_steps, freq='D')
    forecast_df = pd.DataFrame({
        'item id': item_id,
        'ds': forecast_index,
        'forecasted_quantity': forecast.predicted_mean
    })
    forecast_results.append(forecast_df)

forecast_all = pd.concat(forecast_results)

df_inventory = pd.read_csv('inventory_data.csv')

df_inventory['item id'] = df_inventory['item name'].apply(lambda x: int(x.split()[1]))

forecast_merged = forecast_all.merge(df_inventory[['item id', 'stock quantity']], on='item id', how='left')

summary = forecast_merged.groupby('item id').agg({
    'forecasted_quantity': 'sum',
    'stock quantity': 'max'
}).reset_index()

insufficient_inventory = []

for index, row in summary.iterrows():
    item_id = row['item id']
    total_forecasted_demand = row['forecasted_quantity']
    inventory_stock = row['stock quantity']
    
    if inventory_stock < total_forecasted_demand:
        insufficient_inventory.append({
            'item id': item_id,
            'forecasted_demand': total_forecasted_demand,
            'available_stock': inventory_stock
        })

if insufficient_inventory:
    print("Items with insufficient inventory for the forecasted demand:")
    for item in insufficient_inventory:
        print(f"Item ID {item['item id']}: Forecasted Demand = {item['forecasted_demand']:.2f}, Available Stock = {item['available_stock']}")
else:
    print("All items have sufficient inventory for the forecasted demand.")

print("Demand forecasting and inventory check complete.")


All items have sufficient inventory for the forecasted demand.
Demand forecasting and inventory check complete.
